In [1]:
import torch
import numpy as np
from transforms import get_O
from transforms import t_get_O

In [2]:
N=16
L=8
d=2
O_N = get_O(N)
O_L = get_O(L)

In [3]:
z = torch.tensor([[0,0,0],[1,1,1]])
x = torch.reshape(z,(6,))

In [5]:
print(x)

tensor([0, 0, 0, 1, 1, 1])


In [6]:
def reshape(z):
    n_samp = z.shape[0]
    n_nod = z.shape[1]
    dim = z.shape[2]
    z_new = torch.zeros((n_samp,n_nod*dim))
    mask = torch.tensor([dim*j for j in range(n_nod)])
    for i in range(dim):
        z_new[:,mask] = z[:,:,i]
        mask += 1
    return z_new    

In [7]:
def inv_reshape(z,n_nod,dim):
    n_samp = z.shape[0]
    z_new = torch.zeros((n_samp,n_nod,dim))
    mask = torch.tensor([dim*j for j in range(n_nod)])
    for i in range(dim):
        z_new[:,:,i] = z[:,mask]
        mask += 1
    return z_new    

In [8]:
def index_to_multi(k,d,L):
    K = k
    multi = [0] * (d+1)
    for i in range(d):
        multi[i] = K % L
        K = K//L
    multi[d] = K
    return multi
        
    

In [9]:
def multi_to_index(multi,d,L):
    k = sum([multi[j]*L**j for j in range(d+1)])
    return k

In [10]:
def diag_K(L):
    diag = L * [0]
    k = np.arange(1,L//2)
    sin2 = 4 * np.sin(np.pi * k/L)**2
    diag[1:-2:2] = sin2
    diag[2:-1:2] = sin2
    diag[0] = 0
    diag[-1] = 4
    return diag

In [11]:
print(diag_K(32))

[0, np.float64(0.038429439193539104), np.float64(0.038429439193539104), np.float64(0.15224093497742647), np.float64(0.15224093497742647), np.float64(0.3370607753949094), np.float64(0.3370607753949094), np.float64(0.585786437626905), np.float64(0.585786437626905), np.float64(0.8888595339607955), np.float64(0.8888595339607955), np.float64(1.2346331352698203), np.float64(1.2346331352698203), np.float64(1.6098193559677434), np.float64(1.6098193559677434), np.float64(1.9999999999999996), np.float64(1.9999999999999996), np.float64(2.390180644032257), np.float64(2.390180644032257), np.float64(2.7653668647301797), np.float64(2.7653668647301797), np.float64(3.111140466039204), np.float64(3.111140466039204), np.float64(3.414213562373095), np.float64(3.414213562373095), np.float64(3.662939224605091), np.float64(3.662939224605091), np.float64(3.8477590650225735), np.float64(3.8477590650225735), np.float64(3.9615705608064604), np.float64(3.9615705608064604), 4]


In [62]:
big_O = np.ones((N * L ** d,N * L ** d))

In [64]:
for I in range(N*L**d):
    multi_I = index_to_multi(I,d,L)
    print(multi_I)

[0, 0, 0]
[1, 0, 0]
[2, 0, 0]
[3, 0, 0]
[4, 0, 0]
[5, 0, 0]
[6, 0, 0]
[7, 0, 0]
[0, 1, 0]
[1, 1, 0]
[2, 1, 0]
[3, 1, 0]
[4, 1, 0]
[5, 1, 0]
[6, 1, 0]
[7, 1, 0]
[0, 2, 0]
[1, 2, 0]
[2, 2, 0]
[3, 2, 0]
[4, 2, 0]
[5, 2, 0]
[6, 2, 0]
[7, 2, 0]
[0, 3, 0]
[1, 3, 0]
[2, 3, 0]
[3, 3, 0]
[4, 3, 0]
[5, 3, 0]
[6, 3, 0]
[7, 3, 0]
[0, 4, 0]
[1, 4, 0]
[2, 4, 0]
[3, 4, 0]
[4, 4, 0]
[5, 4, 0]
[6, 4, 0]
[7, 4, 0]
[0, 5, 0]
[1, 5, 0]
[2, 5, 0]
[3, 5, 0]
[4, 5, 0]
[5, 5, 0]
[6, 5, 0]
[7, 5, 0]
[0, 6, 0]
[1, 6, 0]
[2, 6, 0]
[3, 6, 0]
[4, 6, 0]
[5, 6, 0]
[6, 6, 0]
[7, 6, 0]
[0, 7, 0]
[1, 7, 0]
[2, 7, 0]
[3, 7, 0]
[4, 7, 0]
[5, 7, 0]
[6, 7, 0]
[7, 7, 0]
[0, 0, 1]
[1, 0, 1]
[2, 0, 1]
[3, 0, 1]
[4, 0, 1]
[5, 0, 1]
[6, 0, 1]
[7, 0, 1]
[0, 1, 1]
[1, 1, 1]
[2, 1, 1]
[3, 1, 1]
[4, 1, 1]
[5, 1, 1]
[6, 1, 1]
[7, 1, 1]
[0, 2, 1]
[1, 2, 1]
[2, 2, 1]
[3, 2, 1]
[4, 2, 1]
[5, 2, 1]
[6, 2, 1]
[7, 2, 1]
[0, 3, 1]
[1, 3, 1]
[2, 3, 1]
[3, 3, 1]
[4, 3, 1]
[5, 3, 1]
[6, 3, 1]
[7, 3, 1]
[0, 4, 1]
[1, 4, 1]
[2, 4, 1]
[3, 4, 1]


In [66]:
for I in range(N*L**d):
    for J in range(N*L**d):
        multi_I = index_to_multi(I,d,L)
        multi_J = index_to_multi(J,d,L)
        for k in range(d):
            big_O[I][J] *= O_L[multi_I[k]][multi_J[k]]
        #print(multi_J[d])    
        big_O[I][J] *= O_N[multi_I[d]][multi_J[d]] 


In [70]:
big_O_kron = np.copy(O_N)
for k in range(d):
    big_O_kron = np.kron(big_O_kron,O_L)

In [72]:
print(big_O-big_O_kron)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [31]:
z = torch.ones((128,64,4))
x = reshape(z)
print(x.shape)
y = inv_reshape(x,64,4)
print(y.shape)

torch.Size([128, 256])
torch.Size([128, 64, 4])


In [ ]:
def get_pair_split_masks(dim):
    mask1 = list(range(0,dim,4))
    mask2 = list(range(2,dim,4))
    mask1 = (mask1+list(map(lambda x:x+1,mask1)))
    mask2 = (mask2+list(map(lambda x:x+1,mask2)))
    mask1.sort()
    mask2.sort()
    split_masks=[mask1,mask2]
    return split_masks

In [ ]:
def get_pair_split_masks(dim):
    bigmask = list(range(dim))
    mask1 = list(filter(lambda x: x%4<2,bigmask))
    mask2 = list(filter(lambda x: x%4>=2,bigmask))
    return mask1, mask2

In [1]:
def get_pair_split_masks_field(latt_size,space_dim,dir):
    bigmask = list(range(L**(space_dim+1)))
    mask1 = list(filter(lambda x: x% (4 * latt_size ** dir) < 2 * latt_size ** dir,bigmask))       
    mask2 = list(filter(lambda x: x%4>=2,bigmask))
    return mask1, mask2

In [3]:
get_pair_split_masks_field(8,1,1)

([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47],
 [2,
  3,
  6,
  7,
  10,
  11,
  14,
  15,
  18,
  19,
  22,
  23,
  26,
  27,
  30,
  31,
  34,
  35,
  38,
  39,
  42,
  43,
  46,
  47,
  50,
  51,
  54,
  55,
  58,
  59,
  62,
  63])

In [ ]:
def configure_flows(n_flows,num_hidden,hidden_dim,p_drop,dim,param_dim=0,mask_config = get_pair_split_masks,sys_dim = 1):
        flows = []
        split_masks_d = mask_config(dim)
    
        for k in range(n_flows):
            theta = ThetaNetwork.configure_theta(num_hidden = num_hidden, 
                                                 hidden_dim = hidden_dim, 
                                                 p_drop = p_drop,
                                                 in_dim = (dim//2 + param_dim) * sys_dim ,
                                                 out_dim = dim//2 * sys_dim)
            flows.append(AffineCouplingLayer(theta, split = split_masks_d, swap = k % 2))
   
        flows = nn.ModuleList(flows)
        return flows     

In [ ]:
def configure_flows_field(n_flows,num_hidden,hidden_dim,p_drop,dim,latt_size,space_dim,mask_config ,param_dim=0):
        flows = []    
        for k in range(n_flows):
            for dir in range(space_dim):
                split_masks = mask_config(latt_size,space_dim,dir)
                theta = ThetaNetwork.configure_theta(num_hidden = num_hidden, 
                                                 hidden_dim = hidden_dim, 
                                                 p_drop = p_drop,
                                                 in_dim = (dim//2 + param_dim) * sys_dim ,
                                                 out_dim = dim//2 * sys_dim)
                flows.append(AffineCouplingLayer(theta, split = split_masks, swap = k % 2))
   
        flows = nn.ModuleList(flows)
        return flows     

SyntaxError: parameter without a default follows parameter with a default (3395937144.py, line 1)

In [21]:
t=torch.tensor([1,2,3,4,5,6,7,8])
t.repeat(1,4)

tensor([[1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, 8,
         1, 2, 3, 4, 5, 6, 7, 8]])

In [3]:
from lattice import Lattice
T = Lattice.get_shift_mat(8)
print(T)

tensor([[0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 0., 0., 0.]])


In [1]:
import torch
import numpy as np
from lattice import Lattice
torch.set_default_dtype(torch.float64)
latt = Lattice([4,4,4],[16,16,16])
O = latt.ort_mat
K = latt.kin_mat
diag = torch.einsum("ji,jk,ks->is",O,K,O)
print(diag.dtype)

torch.float64


In [2]:
import torch
import numpy as np
from lattice import Lattice
latt = Lattice([4,4,4],[16,16,16])
norm_dist = torch.distributions.Normal(loc = torch.zeros(latt.total_nodes), scale = torch.ones(latt.total_nodes))
d = torch.diag(latt.get_diag_kin_mat())

In [3]:
tl = latt.get_train_loader(2**18,2**12)
batch = next(iter(tl))
print(batch.device)

TypeError: Distribution_set.__init__() takes from 2 to 3 positional arguments but 4 were given

In [54]:
n_cols = (N+1)//2-1

column = torch.arange(1,n_cols+1)
line = torch.arange(N)
args = (2*np.pi/N) * torch.einsum('i,j->ij',line,column)

ort = torch.zeros((N,2*n_cols))
ort[:,np.arange(0,2*n_cols,2)] = torch.cos(args)
ort[:,np.arange(1,2*n_cols,2)] = torch.sin(args)
ort = (2/N)**0.5 * ort

first = torch.ones((N,1))
if N%2!=0:
    signed = torch.ones((N,1))
    signed[np.arange(1,N,2),:] = -1
    first = torch.cat((first,signed),1)

ort = torch.cat((first,ort),1)    
print(first)

tensor([[ 1.,  1.],
        [ 1., -1.],
        [ 1.,  1.],
        [ 1., -1.],
        [ 1.,  1.],
        [ 1., -1.],
        [ 1.,  1.],
        [ 1., -1.],
        [ 1.,  1.],
        [ 1., -1.],
        [ 1.,  1.],
        [ 1., -1.],
        [ 1.,  1.],
        [ 1., -1.],
        [ 1.,  1.]])


In [73]:
def get_ort_mat(N,dtype=torch.float64,fmt="torch"):

    n_cols = (N+1)//2-1

    column = torch.arange(1,n_cols+1,dtype=torch.float64)
    line = torch.arange(N,dtype=dtype)
    args = (2*np.pi/N) * torch.einsum('i,j->ij',line,column)

    ort = torch.zeros((N,2*n_cols),dtype=dtype)
    ort[:,np.arange(0,2*n_cols,2)] = torch.cos(args)
    ort[:,np.arange(1,2*n_cols,2)] = torch.sin(args)
    ort = (2/N)**0.5 * ort

    first = torch.ones((N,1),dtype=dtype)
    if N%2==0:
        signed = torch.ones((N,1))
        signed[np.arange(1,N,2),:] = -1
        first = torch.cat((first,signed),1)
    first = first * N ** (-0.5)
    
    ort = torch.cat((first,ort),1)

    if fmt == "np":
        ort = ort.numpy()

    return ort    



In [82]:
from transforms import get_O, t_get_O
o = get_ort_mat(32)
id = torch.einsum("ij,ik->jk",o,o)
print(o.dtype)

torch.float64
